In [1]:
import pandas as pd
import sqlite3

In [2]:
# conectamos con la base de datos my_database.db
connection = sqlite3.connect("my_database.db")

# obtenemos un cursor que utilizamos para las queries
crsr = connection.cursor()

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [4]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

In [5]:
query = '''
CREATE TABLE IF NOT EXISTS reparto (
	CodE VARCHAR(2) NOT NULL,
	CodB VARCHAR(3) NOT NULL,
	CodC VARCHAR(2) NOT NULL,
	Fecha DATE NOT NULL,
	Cantidad SMALLINT,
	PRIMARY KEY (CodE,CodB,CodC,fecha)
);
'''
crsr.execute(query)

In [6]:
query = '''
CREATE TABLE IF NOT EXISTS cervezas (
    CodC VARCHAR(2),
    Envase VARCHAR(32),
    Capacidad FLOAT(2),
    Stock INT(5),
    PRIMARY KEY (CodC)
)
'''

crsr.execute(query)

In [7]:
query = '''
CREATE TABLE IF NOT EXISTS bares (
    CodB VARCHAR(2),
    Nombre VARCHAR(32),
    Cif VARCHAR(32),
    Localidad VARCHAR(32),
    PRIMARY KEY (CodB)
)
'''

crsr.execute(query)

In [8]:
query = '''
CREATE TABLE IF NOT EXISTS empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(32),
    Sueldo INT(5),
    PRIMARY KEY (CodE)
)
'''

crsr.execute(query)

In [9]:
query = '''
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad) 
VALUES  ('1','001','01','2005-10-21',240 ),
( '1','001','02','2005-10-21',48 ),
( '1','002','03','2005-10-22',60 ),
( '1','004','05','2005-10-22',4 ),
( '2','002','03','2005-10-22',48 ),
( '2','002','05','2005-10-23',2 ),
( '2','004','01','2005-10-23',480 ),
( '2','004','02','2005-10-24',72 ),
( '3','003','03','2005-10-24',48 ),
( '3','003','04','2005-10-25',20 )
;
'''
crsr.execute(query)

In [10]:
query = '''
INSERT OR REPLACE INTO bares VALUES ('001','Stop', '11111111X','Villa Botijo'),
('002', 'Las Vegas', '22222222Y', 'Villa Botijo'),
('003', 'Club Social', NULL, 'Las Ranas'),
('004','Otra Ronda','33333333Z','La Esponja')
'''
crsr.execute(query)

In [11]:
query = '''
INSERT OR IGNORE INTO Cervezas VALUES ('01','Botella',0.2,3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04','Botella',1,288),
('05','Barril',60,30)
'''
crsr.execute(query)

In [12]:
query = '''
INSERT OR REPLACE INTO empleados VALUES ('1','Prudencio Caminero',120000),
('2', 'Vicente Merario', 110000),
('3', 'Valentin Siempre', 100000)
'''
# CRUD Create Register(insert) Update Delete
crsr.execute(query)

In [14]:
query = '''
SELECT * FROM reparto 
'''
sql_query(query)

,CodE,CodB,CodC,Fecha,Cantidad
0,1,001,01,2005-10-21,240
1,1,001,02,2005-10-21,48
2,1,002,03,2005-10-22,60
3,1,004,05,2005-10-22,4
4,2,002,03,2005-10-22,48
5,2,002,05,2005-10-23,2
6,2,004,01,2005-10-23,480
7,2,004,02,2005-10-24,72
8,3,003,03,2005-10-24,48
9,3,003,04,2005-10-25,20


In [38]:
# 1 Obtener el nombre de los empleados que hayan repartido al bar Stop durante la semana 
# del 17 al 23 de octubre de 2005.

query = '''
SELECT DISTINCT empleados.Nombre
FROM empleados
INNER JOIN reparto
ON empleados.CodE = reparto.CodE
INNER JOIN bares
ON bares.CodB = reparto.CodB
WHERE bares.nombre = 'Stop' AND reparto.fecha BETWEEN '17/10/2005' AND '23/10/2005'

'''

sql_query(query)

,CodE,Nombre,Sueldo,CodE,CodB,CodC,Fecha,Cantidad,CodB,Nombre,Cif,Localidad
0,1,Prudencio Caminero,120000,1,001,01,2005-10-21,240,001,Stop,11111111X,Villa Botijo
1,1,Prudencio Caminero,120000,1,001,02,2005-10-21,48,001,Stop,11111111X,Villa Botijo


In [46]:
# 2
query = '''
SELECT DISTINCT bares.nombre, bares.cif
FROM bares
INNER JOIN reparto
ON bares.CodB= reparto.CodB
INNER JOIN cervezas
ON cervezas.CodC= reparto.CodC
WHERE cervezas.capacidad <1 AND cervezas.envase = 'Botella'
ORDER BY bares.Localidad
'''

sql_query(query)

,Nombre,Cif
0,Otra Ronda,33333333Z
1,Stop,11111111X


In [47]:
#3
query =('''
SELECT bares.nombre, cervezas.Envase,cervezas.Capacidad, reparto.Fecha,reparto.Cantidad
FROM reparto
INNER JOIN bares
ON reparto.CodB = bares.CodB
INNER JOIN cervezas
ON reparto.CodC = cervezas.CodC
INNER JOIN empleados
ON reparto.codE = empleados.CodE
WHERE empleados.nombre = 'Prudencio Caminero'
''')
sql_query(query)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,2005-10-21,240
1,Stop,Botella,0.33,2005-10-21,48
2,Las Vegas,Lata,0.33,2005-10-22,60
3,Otra Ronda,Barril,60.00,2005-10-22,4


In [49]:
#4
query = '''
SELECT DISTINCT bares.nombre
FROM cervezas
INNER JOIN reparto
ON cervezas.CodC = reparto.CodC
INNER JOIN bares
ON bares.CodB = reparto.CodB
WHERE capacidad IN (0.2, 0.33) AND envase = "Botella"
'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [53]:
#4
query = '''
SELECT DISTINCT b.nombre --, c.envase, c.capacidad

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
WHERE c.envase = 'Botella' AND c.capacidad IN (0.2, 0.33)

'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [54]:
#5
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE
WHERE c.envase = 'Botella' AND b.nombre  IN ("Stop", "Las Vegas")

'''
sql_query(query)

,nombreE,nombreB,Envase
0,Prudencio Caminero,Stop,Botella
1,Prudencio Caminero,Stop,Botella


In [56]:
#5 
query = '''
SELECT e.nombre as nombreE, b.nombre as nombreB, c.envase

FROM bares b
INNER JOIN reparto r
ON b.Codb = r.Codb
INNER JOIN cervezas c
ON r.Codc = c.Codc
INNER JOIN empleadoS e
ON e.CodE = r.CodE

GROUP BY e.nombre 
HAVING c.envase = 'Botella' AND b.nombre IN ('Stop', 'Las Vegas')
'''
sql_query(query)

,nombreE,nombreB,Envase
0,Prudencio Caminero,Stop,Botella


In [71]:
#6
query = '''
SELECT  e.nombre AS nombreE, b.nombre AS nombreB, b.localidad, count(distinct r.fecha)
FROM empleados e
INNER JOIN reparto r
ON e.CodE = r.CodE
INNER JOIN bares b
ON b.CodB = r.CodB
WHERE localidad != 'Villa Botijo'
GROUP BY e.nombre, b.nombre
'''
sql_query(query)

,nombreE,nombreB,Localidad,count(distinct r.fecha)
0,Prudencio Caminero,Otra Ronda,La Esponja,1
1,Valentin Siempre,Club Social,Las Ranas,2
2,Vicente Merario,Otra Ronda,La Esponja,2


In [74]:
# 7
query = '''
SELECT t.nombre, t.localidad, max(t.maximo) AS maximo
FROM

(SELECT 
b.nombre, b.localidad, max(r.cantidad * c.capacidad) as maximo
FROM cervezas c
INNER JOIN reparto r
ON c.CodC = r.CodC
INNER JOIN bares b
ON b.CodB = r.CodB

GROUP BY b.nombre, b.localidad)  t


'''
sql_query(query)

,nombre,localidad,maximo
0,Otra Ronda,La Esponja,240.0


In [81]:
query = '''
SELECT 
b.nombre, b.localidad, r.cantidad, c.capacidad, (r.cantidad * c.capacidad) as maximo
FROM cervezas c
INNER JOIN reparto r
ON c.CodC = r.CodC
INNER JOIN bares b
ON b.CodB = r.CodB

GROUP BY b.nombre, b.localidad, (r.cantidad * c.capacidad)
'''
sql_query(query)

,Nombre,Localidad,Cantidad,Capacidad,maximo
0,Club Social,Las Ranas,48,0.33,15.84
1,Club Social,Las Ranas,20,1.00,20.00
2,Las Vegas,Villa Botijo,48,0.33,15.84
3,Las Vegas,Villa Botijo,60,0.33,19.80
4,Las Vegas,Villa Botijo,2,60.00,120.00
5,Otra Ronda,La Esponja,72,0.33,23.76
6,Otra Ronda,La Esponja,480,0.20,96.00
7,Otra Ronda,La Esponja,4,60.00,240.00
8,Stop,Villa Botijo,48,0.33,15.84
9,Stop,Villa Botijo,240,0.20,48.00


In [83]:
#8
query = '''
SELECT 
DISTINCT b.nombre

FROM bares b
INNER JOIN reparto r
ON b.CodB= r.CodB
INNER JOIN cervezas c
ON c.CodC = r.CodC
WHERE c.envase = 'Botella' AND c.Capacidad <1

'''
sql_query(query)

,Nombre
0,Stop
1,Otra Ronda


In [ ]:
query =('''
SELECT t.Nombre, max(t.dias) dias, t.Sueldo*1.05 nuevo_sueldo
FROM 
(SELECT e.nombre, count(Distinct r.fecha) AS dias, e.Sueldo
FROM reparto r
INNER JOIN bares b
ON r.CodB = b.CodB
INNER JOIN empleados e
ON r.codE = e.CodE
GROUP BY e.nombre) t
''')
sql_query(query)

In [ ]:
query9 = '''
SELECT t.Sueldo*1.05, MAX(t.maximo) ,t.Nombre
FROM (SELECT e.Sueldo, COUNT(DISTINCT r.Fecha) as maximo, e.Nombre
    FROM empleados as e
    INNER JOIN reparto as r
    ON e.CodE = r.CodE
    GROUP BY e.Nombre) t
'''
sql_query(query9)

In [ ]:
query = '''
UPDATE empleados
SET sueldo = sueldo * 1.05
WHERE CodE IN (
SELECT 
CodE
FROM reparto
GROUP BY CodE
ORDER BY COUNT( DISTINCT Fecha) DESC LIMIT 1)
'''
crsr.execute(query)